# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [57]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests as rq

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,32.58,68,42,13.31,TF,Fri Jul 19 15:22:45 2024
1,1,ushuaia,-54.8000,-68.3000,46.02,81,40,4.61,AR,Fri Jul 19 15:22:45 2024
2,2,igarka,67.4667,86.5833,45.19,67,4,14.58,RU,Fri Jul 19 15:22:45 2024
3,3,waitangi,-43.9535,-176.5597,52.83,95,100,28.83,NZ,Fri Jul 19 15:22:45 2024
4,4,ilulissat,69.2167,-51.1000,48.47,63,63,3.76,GL,Fri Jul 19 15:22:46 2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    size= 'Humidity',
    scale = 1,
    color = 'City',
    alpha = 0.5,
    tiles = 'OSM',
    frame_width=700,
    frame_height=500
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
# Display sample data
city_data_df[
    (city_data_df['Max Temp']>=70)&
    (city_data_df['Max Temp']<=75)&
    (city_data_df['Humidity']<=70)&
    (city_data_df['Cloudiness']<=10)&
    (city_data_df['Wind Speed']<=10)
]



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
104,104,uzgen,40.7699,73.3007,70.09,55,5,6.35,KG,Fri Jul 19 15:23:08 2024
260,260,kirando,-7.4167,30.6000,73.31,47,0,6.06,TZ,Fri Jul 19 15:23:40 2024
277,277,lata,40.1629,-8.3327,74.91,69,1,6.24,PT,Fri Jul 19 15:23:44 2024
345,345,vila franca do campo,37.7167,-25.4333,73.81,66,1,5.46,PT,Fri Jul 19 15:23:59 2024
382,382,kalach,50.4258,41.0261,72.36,61,7,9.48,RU,Fri Jul 19 15:24:06 2024
522,522,malanje,-9.5402,16.3410,72.91,14,3,3.76,AO,Fri Jul 19 15:24:37 2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[
    (city_data_df['Max Temp']>=70)&
    (city_data_df['Max Temp']<=75)&
    (city_data_df['Humidity']<=70)&
    (city_data_df['Cloudiness']<=10)&
    (city_data_df['Wind Speed']<=10)
]
hotel_df = hotel_df.drop(['City_ID', 'Max Temp', 'Wind Speed', 'Date'],axis=1).copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Cloudiness,Country,Hotel Name
104,uzgen,40.7699,73.3007,55,5,KG,
260,kirando,-7.4167,30.6000,47,0,TZ,
277,lata,40.1629,-8.3327,69,1,PT,
345,vila franca do campo,37.7167,-25.4333,66,1,PT,
382,kalach,50.4258,41.0261,61,7,RU,
522,malanje,-9.5402,16.3410,14,3,AO,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [58]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # 
    lat = row.Lat
    lon = row.Lng

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = rq.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
uzgen - nearest hotel: Кунрат
kirando - nearest hotel: Lakeshore Lodge
lata - nearest hotel: Residencial Botânico
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
kalach - nearest hotel: No hotel found
malanje - nearest hotel: Hotel PORTVGALIA


,City,Lat,Lng,Humidity,Cloudiness,Country,Hotel Name
104,uzgen,40.7699,73.3007,55,5,KG,Кунрат
260,kirando,-7.4167,30.6000,47,0,TZ,Lakeshore Lodge
277,lata,40.1629,-8.3327,69,1,PT,Residencial Botânico
345,vila franca do campo,37.7167,-25.4333,66,1,PT,Vinha d'Areia Beach Hotel
382,kalach,50.4258,41.0261,61,7,RU,No hotel found
522,malanje,-9.5402,16.3410,14,3,AO,Hotel PORTVGALIA


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    size= 'Humidity',
    scale = 1,
    color = 'City',
    alpha = 0.5,
    tiles = 'OSM',
    hover_cols = ['Hotel Name', 'Country'],
    frame_width=700,
    frame_height=500
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)